In [1]:
import django_initializer
from telemetry.fast_lap_analyzer import FastLapAnalyzer
from telemetry.racing_stats import RacingStats
from telemetry.models import FastLap, Lap
from telemetry.analyzer import Analyzer
from telemetry.influx import Influx
import plotly.io as pio

# pio.renderers.default = "svg"  # comment this line to use interactive plots

import pandas as pd
import numpy as np

from telemetry.visualizer import *

%load_ext autoreload
%autoreload 2

analyzer = Analyzer()
influx = Influx()
fast_lap_analyzer = FastLapAnalyzer()
racing_stats = RacingStats()

2023-07-19 20:03:27,763 DEBUG Influx: Connected to https://telemetry.b4mad.racing/


In [2]:
kwargs = {
    # "game": "iRacing",
    "game": "Assetto Corsa Competizione",
    # "track": "oschersleben gp",
    # "track": "aragon gp",
    # "track": "jerez gp",
    # "track": "spa up",
    "track": "Hungaroring:track config",
    # "car": "Ferrari 488 GT3 Evo 2020",
    "car": "GT3",
    "valid": True,
}
laps = racing_stats.laps(**kwargs)
laps = laps[:10]
# laps = list(laps)
fast_lap_analyzer.laps = laps
data_frames, laps_with_telemetry = fast_lap_analyzer.fetch_lap_telemetry()

2023-07-19 20:03:27,856 DEBUG Influx: Connected to https://telemetry.b4mad.racing/
2023-07-19 20:03:27,860 INFO Fetching telemetry for Assetto Corsa Competizione - Hungaroring:track config - GT3
2023-07-19 20:03:27,861 INFO   track.id 32 car.id 3
2023-07-19 20:03:27,861 INFO   session 1683922552 lap.id 54825 number 6
2023-07-19 20:03:27,861 INFO   length 4374 time 105.72 valid True
2023-07-19 20:03:27,862 INFO   start 2023-05-12 21:27:25.270823+00:00 end 2023-05-12 21:29:10.860304+00:00
2023-07-19 20:03:28,083 INFO Fetching telemetry for Assetto Corsa Competizione - Hungaroring:track config - GT3
2023-07-19 20:03:28,084 INFO   track.id 32 car.id 3
2023-07-19 20:03:28,084 INFO   session 1683909448 lap.id 54357 number 5
2023-07-19 20:03:28,084 INFO   length 4377 time 105.887 valid True
2023-07-19 20:03:28,085 INFO   start 2023-05-12 17:45:43.104091+00:00 end 2023-05-12 17:47:28.988500+00:00
2023-07-19 20:03:28,265 INFO Fetching telemetry for Assetto Corsa Competizione - Hungaroring:track

In [3]:
# influx_data_frames = []
# for df in data_frames:
#     influx_data_frames.append(df.copy())

# data_frames = []
# for df in influx_data_frames:
#     data_frames.append(df.copy())

In [4]:
current_sectors = fast_lap_analyzer.current_fast_lap_sectors()
display(current_sectors)

[{'start': 918, 'end': 1631},
 {'start': 1632, 'end': 1856},
 {'start': 1857, 'end': 2207},
 {'start': 2208, 'end': 2442},
 {'start': 2443, 'end': 2930},
 {'start': 2931, 'end': 3338},
 {'start': 3339, 'end': 3613},
 {'start': 3614, 'end': 3876},
 {'start': 3877, 'end': 4374}]

In [7]:
# for i, df in enumerate(data_frames):
#     fig = lap_fig(df, full_range=True, columns=["Throttle"])
#     fig.show()
sector_start_end, df_max = fast_lap_analyzer.extract_sectors(data_frames)
display(sector_start_end)

# are the sectors similar to the current fast lap?
similarity = fast_lap_analyzer.similar_sectors(sector_start_end, current_sectors)
display(f"Sectors are similar: {similarity}")

fig = lap_fig(df_max, full_range=True, columns=["Throttle", "Brake"])
# add a vertical line for each sector
for i, sector in enumerate(sector_start_end):
    fig_add_shape(fig, x0=sector["start"], x1=sector["start"], color="blue")
    fig_add_shape(fig, x0=sector["end"], x1=sector["end"], color="red")
fig.show()

2023-07-19 20:11:04,022 DEBUG split_sectors: min_length_throttle_below_threshold: 20
2023-07-19 20:11:04,023 DEBUG split_sectors: min_distance_between_sectors: 35
2023-07-19 20:11:04,024 DEBUG sector 2 - 1149m too short: 11.0
2023-07-19 20:11:04,024 DEBUG sector 5 - 2076m too short: 15.0
2023-07-19 20:11:04,024 DEBUG sector 13 - 3829m too short: 1.0
2023-07-19 20:11:04,025 DEBUG sector 15 - 4066m too short: 6.0
2023-07-19 20:11:04,025 DEBUG sector 17 - 4109m too short: 4.0
2023-07-19 20:11:04,025 DEBUG sector 18 - 4122m too short: 1.0
2023-07-19 20:11:04,025 DEBUG remove sector 10 - 3790m too close to previous sector: 4.0
2023-07-19 20:11:04,025 DEBUG remove sector 12 - 4077m too close to previous sector: 21.0


[{'start': 0, 'end': 921.0, 'length': 921},
 {'start': 922.0, 'end': 1633.0, 'length': 711},
 {'start': 1634.0, 'end': 1862.0, 'length': 228},
 {'start': 1863.0, 'end': 2207.0, 'length': 344},
 {'start': 2208.0, 'end': 2442.0, 'length': 234},
 {'start': 2443.0, 'end': 2644.0, 'length': 201},
 {'start': 2645, 'end': 2929.0, 'length': 284},
 {'start': 2930.0, 'end': 3329.0, 'length': 399},
 {'start': 3330.0, 'end': 3613.0, 'length': 283},
 {'start': 3614.0, 'end': 3889.0, 'length': 275},
 {'start': 3890.0, 'end': 4374.25537, 'length': 484}]

'Sectors are similar: False'

In [8]:
segments, used_laps = fast_lap_analyzer.extract_segments(sector_start_end, data_frames, laps_with_telemetry, df_max)
track_length = df_max["DistanceRoundTrack"].max()
for i in range(len(segments)):
    segment = segments[i]
    # display(segment.start, segment.end, segment.turn)
    # display(segment.telemetry)
    # df = telemetry_for_fig(segment)
    df = segment.telemetry
    fig = lap_fig(df, columns=["Throttle", "Brake"])
    brake_features = segment.brake_features()
    if brake_features:
        # brake_features = features_for_fig(segment, track_length, brake_features)
        fig_add_features(fig, brake_features)
    throttle_features = segment.throttle_features()
    if throttle_features:
        # throttle_features = features_for_fig(segment, track_length, throttle_features)
        fig_add_features(fig, throttle_features, color="green")
    display(f"time: {segment.time} {segment.start} -> {segment.end} / track: {segment.track_length}")
    fig.show()

2023-07-19 20:11:08,342 DEBUG extract_segments for sector 0 start: 0 end: 921.0
2023-07-19 20:11:08,355 DEBUG extract_segments for sector 1 start: 922.0 end: 1633.0
2023-07-19 20:11:08,365 DEBUG extract_segments for sector 2 start: 1634.0 end: 1862.0
2023-07-19 20:11:08,376 DEBUG extract_segments for sector 3 start: 1863.0 end: 2207.0
2023-07-19 20:11:08,387 DEBUG extract_segments for sector 4 start: 2208.0 end: 2442.0
2023-07-19 20:11:08,399 DEBUG extract_segments for sector 5 start: 2443.0 end: 2644.0
2023-07-19 20:11:08,410 DEBUG extract_segments for sector 6 start: 2645 end: 2929.0
2023-07-19 20:11:08,421 DEBUG extract_segments for sector 7 start: 2930.0 end: 3329.0
2023-07-19 20:11:08,433 DEBUG extract_segments for sector 8 start: 3330.0 end: 3613.0
2023-07-19 20:11:08,442 DEBUG extract_segments for sector 9 start: 3614.0 end: 3889.0
2023-07-19 20:11:08,453 DEBUG extract_segments for sector 10 start: 3890.0 end: 4374.25537


'time: 18.69 0 -> 921 / track: 4374.25537'

'time: 16.205000000000002 922 -> 1633 / track: 4374.25537'

'time: 4.077999999999996 1634 -> 1862 / track: 4374.25537'

'time: 9.190000000000005 1863 -> 2207 / track: 4374.25537'

...

'time: 7.409999999999997 2645 -> 2929 / track: 4374.25537'

'time: 8.177000000000007 2930 -> 3329 / track: 4374.25537'

'time: 6.917999999999992 3330 -> 3613 / track: 4374.25537'

'time: 8.409999999999997 3614 -> 3889 / track: 4374.25537'

'time: 12.555000000000007 3890 -> 4374 / track: 4374.25537'